In [38]:
import pandas as pd
import numpy as np

data = [[1, 'Arsenal'], [2, 'Arsenal'], [3, 'Arsenal'], [4, 'Chelsea'], [5, 'Chelsea'], [6, 'Chelsea']]
teams = pd.DataFrame(data, columns=["player_id", "team_name"]).astype({"player_id": "int", "team_name": "string"})
display(teams)
data = [[1, '00:15', 2], [2, '00:45', 3], [3, '01:15', 1], [4, '00:30', 1], [2, '46:00', 3], [3, '46:15', 4], [1, '46:45', 2], [5, '46:30', 6]]
passes = pd.DataFrame(data, columns=["pass_from", "time_stamp", "pass_to"]).astype({"pass_from": "int", "time_stamp": "string", "pass_to": "int"})
display(passes)

,player_id,team_name
0,1,Arsenal
1,2,Arsenal
2,3,Arsenal
3,4,Chelsea
4,5,Chelsea
5,6,Chelsea


,pass_from,time_stamp,pass_to
0,1,00:15,2
1,2,00:45,3
2,3,01:15,1
3,4,00:30,1
4,2,46:00,3
5,3,46:15,4
6,1,46:45,2
7,5,46:30,6


In [39]:
passes["min"] = pd.to_numeric(passes["time_stamp"].str[:2])
passes["sec"] = pd.to_numeric(passes["time_stamp"].str[3:])
passes["half_number"] = np.where((passes["min"]<=45), 1, 2)
passes["half_number"] = np.where((passes["min"]==45) & (passes["sec"]>0), 2, passes["half_number"])
passes = passes.drop(columns=["time_stamp"])

passes = passes.merge(teams, left_on="pass_from", right_on="player_id", how="left")
passes = passes.drop(columns=["player_id", "pass_from"])
passes = passes.rename(columns={"team_name": "pass_from"})

passes = passes.merge(teams, left_on="pass_to", right_on="player_id", how="left")
passes = passes.drop(columns=["player_id", "pass_to"])
passes = passes.rename(columns={"team_name": "pass_to"})

passes["dominance"] = np.where(passes["pass_from"]==passes["pass_to"], 1, -1)
display(passes)

df = passes.groupby(["pass_from", "half_number"])["dominance"].sum().reset_index()
df = df.rename(columns={"pass_from": "team_name"})
df = df.sort_values(by=["team_name", "half_number"])
display(df.head())

,min,sec,half_number,pass_from,pass_to,dominance
0,0,15,1,Arsenal,Arsenal,1
1,0,45,1,Arsenal,Arsenal,1
2,1,15,1,Arsenal,Arsenal,1
3,0,30,1,Chelsea,Arsenal,-1
4,46,0,2,Arsenal,Arsenal,1
5,46,15,2,Arsenal,Chelsea,-1
6,46,45,2,Arsenal,Arsenal,1
7,46,30,2,Chelsea,Chelsea,1


,team_name,half_number,dominance
0,Arsenal,1,3
1,Arsenal,2,1
2,Chelsea,1,-1
3,Chelsea,2,1
